### Introduction:
This notebook corresponds to Chapter 2. The primary objective of this notebook is to provide practical code examples that complement the explanations given in the chapter. Here, we will delve into the different components of a transaction executed on the Ethereum blockchain.

By following the provided steps, you will develop a comprehensive understanding of each part of a transaction and learn how to access and analyze them. This hands-on approach will enhance your proficiency in working with transaction data and enable you to navigate and interpret Ethereum transactions effectively.

#### Imports

Import the necesary libraries to run the notebook.

In [1]:
from web3 import Web3
from datetime import datetime
import pandas as pd
import json

#### Connection
Connect to the Ethereum blockchain using the web3 library and the Infura service. Check if the connection is successful by calling the `web3.isConnected()` method.

In [2]:
infura_url= 'https://mainnet.infura.io/v3/[YOUR API KEY]'
web3= Web3(Web3.HTTPProvider (infura_url))
web3.isConnected()

True

# Dissecting Transaction

We will define one address and one transaction to use as example. 

In [3]:
address= web3.toChecksumAddress("0x1ad91ee08f21be3de0ba2ba6918e714da6b45836")
transaction= '0x032ed60363beb809a2b9c9790bb7dadd83b743040945a087aeecbe9e6b2dc2af'

* Nonce

#### Retrieve the Account nonce

We can query directly the address and it retrieve the number up to the latest block.

In [4]:
next_nonce= web3.eth.getTransactionCount(address)
next_nonce

3460174

Or we can query a transaction, that will provide the situation of the source address at the moment of that transaction.  

In [5]:
transaction_nonce= web3.eth.get_transaction(transaction).nonce
transaction_nonce

3460171

* Gas

#### Retrieve Transaction Gas Price
Retrieve the gas price for a given transaction using the `web3.eth.get_transaction()` method specifying the `gas price` property. 

In [6]:
gasPrice= web3.eth.get_transaction(transaction).gasPrice
gasPrice

19209576968

#### Retrieve Transaction Gas limit
Retrieve the gas price for a given transaction using the `web3.eth.get_transaction()` method specifying the `gas` property. 

In [7]:
gaslimit= web3.eth.get_transaction(transaction).gas
gaslimit

100000

#### Retrieve Transaction Gas Used
Retrieve the gas price for a given transaction using the `web3.eth.get_transaction()` method specifying the `gasUsed` property. 

In [8]:
gasUsed= web3.eth.getTransactionReceipt (transaction).gasUsed
gasUsed

21000

* To

#### Retrieve Transaction Recipient
Retrieve the transaction recipient for a given transaction using the `web3.eth.get_transaction()` method specifying the `to` property. 

In [9]:
recipient= web3.eth.get_transaction(transaction).to
recipient

'0x3c16183c1C0E28F1a0cB9F8ee4b21D0Db208cA46'

* From

#### Retrieve Transaction Sender
Retrieve the transaction sender for a given transaction using the `web3.eth.get_transaction()` method specifying the `gas price` property. 

In [ ]:
sender= web3.eth.get_transaction(transaction)["from"]
sender

* Value

#### Retrieving Transaction Value
Retrieve the value transferred in a given transaction using the `web3.eth.get_transaction()` method specifying the `value` property.  The value  of the transaction is expressed in wei, which can be converted to ether by multiplying it by `10**-18` or with `web3.fromWei('value','ether')`

In [10]:
value= web3.eth.get_transaction(transaction).value
value*10**-18

48.0

* Data

#### Retrieve Transaction Input
Retrieve the input data in a given transaction using the `web3.eth.get_transaction()` method specifying the `input` property.

In [11]:
data= web3.eth.get_transaction(transaction).input
data

'0x'

#### Decode Transaction Input

We will connect the ABI, the smart contract address and the input data resulting of using the `web3.eth.get_transaction()` method. 

In [12]:
transaction_ba='0xb3827bb3cca1a693ec69edb744755f64d8ff8c90f89f69cbfbfafd17b0083159'

In [13]:
data_ba= web3.eth.get_transaction(transaction_ba).input
data_ba

'0xa723533e0000000000000000000000000000000000000000000000000000000000000001'

In [14]:
bored_ape= web3.eth.get_transaction(transaction_ba)

In [15]:
sc_address="0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D"
with open ("./ba_abi.json") as f:
    abi= json.load (f)

In [16]:
baContract = web3.eth.contract(address=sc_address, abi=abi)

The method `decode_function_input ()` will decode the function input from the bored_ape dictionary. 

In [17]:
function, parameters = baContract.decode_function_input(bored_ape["input"])
print (function, 'parameters: ',parameters )

<Function mintApe(uint256)> parameters:  {'numberOfTokens': 1}


* Log events

#### Decoding Events
Create a list of events listed in the the object baContract. Loop through the abi_events list and create a dictionary called event_abi_hex. The dictionary will contain the event name as the key and the event ABI in hexadecimal format as the value. 

In [18]:
abi_events = [abi for abi in baContract.abi if abi["type"] == "event"]
event_abi_hex = {}
for event in abi_events:
    name = event["name"]
    inputs = ",".join([param["type"] for param in event["inputs"]])
    event_abi_human = f"{name}({inputs})"
    event_abi_hex[name] = web3.toHex(web3.keccak(text=event_abi_human))

Retrieve the transaction receipt for the transaction_ba and loop through the logs in the receipt. For each log, it will get the event in hexadecimal format from the first topic and then compare it to the event_abi_hex dictionary. If a match is found, it will decode the log using `processReceipt(tx_receipt)` 

In [19]:
tx_receipt = web3.eth.getTransactionReceipt(transaction_ba)
decoded_logs = []
for log in tx_receipt['logs']:
    event_hex = web3.toHex(log['topics'][0])
    event_name = None
    for name, hex_value in event_abi_hex.items():
        if hex_value == event_hex:
            event_name = name
            break
    if event_name is not None:
        decoded_log = baContract.events[event_name]().processReceipt(tx_receipt)
        decoded_logs.append(decoded_log)
decoded_logs

(AttributeDict({'args': AttributeDict({'from': '0x0000000000000000000000000000000000000000',
   'to': '0x9909017A0F637380af916257D05c3e7dD2F6c68a',
   'tokenId': 6633}),
  'event': 'Transfer',
  'logIndex': 124,
  'transactionIndex': 321,
  'transactionHash': HexBytes('0xb3827bb3cca1a693ec69edb744755f64d8ff8c90f89f69cbfbfafd17b0083159'),
  'address': '0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D',
  'blockHash': HexBytes('0x26537966b001e26c3c960bfb619d1af5fdd4d04c54596fb50a3a94d73bf66a69'),
  'blockNumber': 12347114}),)